In [1]:
#! usr/bin/env python3
from tqdm import tqdm_notebook as tqdm
import praw
import pandas as pd
import datetime as dt
from PIL import Image
import urllib.request
import wikipedia
import feather
import random
import shutil

import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

import re
import glob
import time

import ssl

ssl._create_default_https_context = ssl._create_unverified_context

import IPython.display as ipd
import pyttsx3
import os
import uuid
import operator

from shutil import copyfile

from pydub import AudioSegment
from google.cloud import texttospeech

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/anthonycelio/xx/My_Projectectect-ef53cc7c18c4.json"

In [3]:
reddit = praw.Reddit(client_id='ZWkdQHV89SHfEw', \
                     client_secret='DE9g8daIEcx7SEH3seN41aDKmIs', \
                     user_agent='Chester', \
                     username='user_00000000000001', \
                     password='Russianriver987')

subreddit = reddit.subreddit('Unexpected')
#subreddit = reddit.subreddit('cringeanarchy')

#top_subreddit = subreddit.hot()
top_subreddit = subreddit.top('week')

#top_subreddit = subreddit.top(limit=50)
#for submission in subreddit.hot(limit=1):
#    print(submission.title, submission.id)
    
topics_dict = { "title":[],
               "score":[],
               "id":[],
               "url":[],
               "comms_num":[],
               "created": [],
               "body":[],
               "vid":[]
                  }

for submission in top_subreddit:
    topics_dict["title"].append(submission.title)
    topics_dict["score"].append(submission.score)
    topics_dict["id"].append(submission.id)
    topics_dict["url"].append(submission.url)
    topics_dict["comms_num"].append(submission.num_comments)
    topics_dict["created"].append(submission.created)
    topics_dict["body"].append(submission.selftext)
    topics_dict["vid"].append(submission.media)
    
topics_data = pd.DataFrame(topics_dict)

In [ ]:
#submission.media['reddit_video']['fallback_url']

In [ ]:
topics_data

In [ ]:
topics_data['comms_num'][5]

In [ ]:
brk = 0
use = []
for i in range(100):
    if topics_data["comms_num"][i] >= 13:
        if topics_data["vid"][i] != None:
            try:
                xex = topics_data["vid"][i]["reddit_video"]["duration"]
                if xex > 4 and xex < 30:
                    use.append(i)
                    brk += 1
            except:
                pass
    
    #if brk >= 5:
     #   break

In [ ]:
len(use)

In [ ]:
z99 = 0
for z, i in enumerate(topics_data['url']):
    if z99 < 10:
        if i[-3:] == 'jpg':
            urllib.request.urlretrieve(i, i[-10:])
            z99 += 1
            print(z, i)

In [ ]:
topics_data.loc[0]

In [ ]:
!pwd

In [ ]:
def ret_com(num):
    po = topics_data['id'][num]
    submission = reddit.submission(id=po)
    submission.comments.replace_more(limit=1)
    chop = []
    
    for top_level_comment in submission.comments:
        chop.append([top_level_comment.body])
        for i in (top_level_comment.replies):
            chop.append([i.body])
            for x in (i.replies):
                chop.append([x.body])
                for z in (x.replies):
                    chop.append([z.body])
                    for cc in (z.replies):
                        chop.append([cc.body])
                        for wo in (cc.replies):
                            chop.append([wo.body])
                            for zoo in (wo.replies):
                                chop.append([zoo.body])

    grab = []
    for i in chop:
        if len(i[0]) > 10:
            if len(i[0]) < 300:
                for x in i:
                    i = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', x)
                    i = re.sub(r'[^a-zA-Z1-9.,()\'"?! ]+', '', i)
                    i = i.replace('ITAP', 'I took a picture')

                    grab.append(i)    
    #[topics_data['title'][num]] + grab
    frst = topics_data['title'][num].replace('ITAP of ', '')
    grab.insert(0, frst.replace('ITAP ', ''))

    return grab[:24]

In [ ]:
cc = ret_com(1)

In [ ]:
cc

In [ ]:
len(cc)

In [ ]:
if cc[-1:][0][-1:] == '?':
    cc = cc[:-1]
if cc[-1:][0][-2:] == '?':
    cc = cc[:-1]

In [ ]:
cc[:-1]

In [ ]:
def TTS_lena(str, wave_name):
    client = texttospeech.TextToSpeechClient()
    synthesis_input = texttospeech.types.SynthesisInput(text=str)

    voice = texttospeech.types.VoiceSelectionParams(
        language_code='en-AU',
        name='en-AU-Wavenet-A',
        ssml_gender=texttospeech.enums.SsmlVoiceGender.FEMALE)

    audio_config = texttospeech.types.AudioConfig(
        speaking_rate=0.91,
        pitch=0,
        audio_encoding=texttospeech.enums.AudioEncoding.MP3)
    response = client.synthesize_speech(synthesis_input, voice, audio_config)
    with open('output.mp3', 'wb') as out:
        out.write(response.audio_content)
    newAudio = AudioSegment.from_mp3('output.mp3')
    newAudio.export(wave_name + '.wav', format="wav")
    os.remove('output.mp3')

In [ ]:
def TTS_stac(str, wave_name):
    client = texttospeech.TextToSpeechClient()
    synthesis_input = texttospeech.types.SynthesisInput(text=str)

    voice = texttospeech.types.VoiceSelectionParams(
        language_code='en-US',
        name='en-US-Wavenet-F',
        ssml_gender=texttospeech.enums.SsmlVoiceGender.FEMALE)

    audio_config = texttospeech.types.AudioConfig(
        speaking_rate=0.89,
        pitch=-1,
        audio_encoding=texttospeech.enums.AudioEncoding.MP3)
    response = client.synthesize_speech(synthesis_input, voice, audio_config)
    with open('output.mp3', 'wb') as out:
        out.write(response.audio_content)
    newAudio = AudioSegment.from_mp3('output.mp3')
    newAudio.export(wave_name + '.wav', format="wav")
    os.remove('output.mp3')

In [ ]:
def wav_2_phone(wav_file):
    new = AudioSegment.from_wav(wav_file)
    new = new.apply_gain(12).low_pass_filter(30000).high_pass_filter(30000)
    new = new + 15
    new.export(wav_file, format="wav")

In [ ]:
wdir = '/Users/anthonycelio/Documents/deep_news/!machine/corespondants/16/'
if not os.path.exists(wdir): os.makedirs(wdir)

In [ ]:
(use[10:14])

In [ ]:
for m in tqdm(range(len(use[10:15]))):
    t = str(int(time.time()))
    sha = wdir+t+'/'
    os.makedirs(sha)
    os.chdir(sha)

    cc = ret_com(use[10:15][m])[:20]
    if cc[-1:][0][-1:] == '?':
        cc = cc[:-1]
    if cc[-1:][0][-2:] == '?':
        cc = cc[:-1]
    mp4s = topics_data["vid"][use[10:15][m]]["reddit_video"]["fallback_url"]
    urllib.request.urlretrieve(mp4s, mp4s[-14:]+'.mp4')
    for z, i in enumerate(tqdm(cc)):
        if z % 2 == 0:
            TTS_lena(i, (str(z)).zfill(2))
            wav_2_phone((str(z)).zfill(2)+'.wav')
        else:
            TTS_stac(i, (str(z)).zfill(2))

In [ ]:
su = '''The Parasol (also known as El Quitasol) is one of a cartoon series of oil on linen paintings made by the painter Francisco Goya. This series of paintings was specifically made in order to be transformed into tapestries that would be hung on the walls of the Royal Palace of El Pardo in Madrid, Spain. The tapestries showed serene events in everyday life, which made them a nice addition to the dining room of Prince and Princess of Asturias—the future King Charles IV and Maria Luisa of Parma. The queen called on Goya because she wanted to decorate the dining room with cheerful scenes; The Parasol and the other tapestry paintings were Goya's response to this request. The painting is currently located in the Museo del Prado in Madrid as is another in the series, Blind man's bluff. '''

In [ ]:
TTS_stac(su, '/Users/anthonycelio/Desktop/goya')

In [ ]:
sea = [int(i) for i in os.listdir(wdir) if len(i) is 10]

In [ ]:
sea

In [ ]:
print('''var vs = '''+'"'+str(wdir)+'"'+'''
var myblocnFolder = new Folder(vs).getFiles()
lena = '''+str(sea))

In [ ]:
wav_2_phone((str(z)).zfill(2)+'.wav')

In [ ]:
lena = '/Users/anthonycelio/Documents/deep_news/correspondants/lena/'

In [ ]:
os.chdir(lena)

In [ ]:
TTS_stac('Welcome back. Joining us now on the phone, our correspondant Lena Bronsky. How are you, Lena?', lena+'int_lena1')

In [ ]:
TTS_lena('Fine Stacy, How are you?', lena+'lena_r1')
wav_2_phone(lena+'lena_r1'+'.wav')

In [ ]:
TTS_stac('''Great. I understand you have some photos for us. What's your first image?''', lena+'St_go_image')

In [ ]:
TTS_stac('''Okay, What's next, Lena?''', lena+'St_nextQ')

In [ ]:
TTS_stac('''and this concludes our broadcast. Thanks for watching.''', lena+'St_show_end' )

In [ ]:
//*[@id="content"]/article/div[4]/div[1]/p/text()

In [16]:
!pip install Wikipedia-API

You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
TTS_lena('Thank you for having me, Stacy.', lena+'lena_out')
wav_2_phone(lena+'lena_out'+'.wav')

In [17]:
import wikipediaapi


In [28]:
wiki_wiki = wikipediaapi.Wikipedia('en')

page = wiki_wiki.page('The_Temptation_of_St_Anthony_(Bosch_painting)')

In [29]:
page

The_Temptation_of_St_Anthony_(Bosch_painting) (id: ??, ns: 0)

In [31]:
page.text

'The Temptation of St. Anthony is a painting of disputed authorship, attributed to either Hieronymus Bosch or a follower. It is now in the Museo del Prado in Madrid.\n\nHistory\nThe work was in the Escorial monastery, although it was not mentioned in inventories; later it was moved to the Prado. It is likely that the work was one of the Temptations sent to the monastery by Philip II of Spain in 1574. This painting has a more serene atmosphere than the triptych with the same theme now in the Museu Nacional de Arte Antiga of Lisbon.Like all Bosch\'s works, it cannot be dated with precision, although it is likely from his late production (1500–1516).In 2016, the Bosch Research and Conservation project, after five years of researching all known Bosch paintings, announced that they had significant doubts about the attribution of the work to Bosch, instead attributing it to a follower.\n\nDescription\nSt. Anthony the Abbot is portrayed in meditation, in a sunny landscape near the trunk of a 

In [26]:
wiki_wiki.page.

In [47]:
wiki_wiki = wikipediaapi.Wikipedia(
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI
)

p_wiki = wiki_wiki.page("The_Headless_Horseman_Pursuing_Ichabod_Crane")
p_wiki.text

'The Headless Horseman Pursuing Ichabod Crane (1858) is a painting by American artist John Quidor, depicting a scene from Washington Irving\'s short story "The Legend of Sleepy Hollow".\nThe schoolmaster Ichabod Crane is fleeing on a white horse, pursued by the Headless Horseman on a black horse.  In one hand, the Headless Horseman is holding a pumpkin, which he is preparing to throw at Crane.  Visible in the background is the Old Dutch Church of Sleepy Hollow and the adjacent Sleepy Hollow Cemetery, and the full moon.\nThe painting is one of several by the artist based on scenes from Irving\'s written works about Dutch New York, including such paintings as Ichabod Crane Flying from the Headless Horseman (1828, Yale University Art Gallery) and The Return of Rip Van Winkle (1849, National Gallery of Art).Quidor first exhibited the picture at the National Academy of Design in New York City. It is currently displayed in the Smithsonian American Art Museum in Washington, D.C., which bought

In [49]:
print(p_wiki.text)

The Headless Horseman Pursuing Ichabod Crane (1858) is a painting by American artist John Quidor, depicting a scene from Washington Irving's short story "The Legend of Sleepy Hollow".
The schoolmaster Ichabod Crane is fleeing on a white horse, pursued by the Headless Horseman on a black horse.  In one hand, the Headless Horseman is holding a pumpkin, which he is preparing to throw at Crane.  Visible in the background is the Old Dutch Church of Sleepy Hollow and the adjacent Sleepy Hollow Cemetery, and the full moon.
The painting is one of several by the artist based on scenes from Irving's written works about Dutch New York, including such paintings as Ichabod Crane Flying from the Headless Horseman (1828, Yale University Art Gallery) and The Return of Rip Van Winkle (1849, National Gallery of Art).Quidor first exhibited the picture at the National Academy of Design in New York City. It is currently displayed in the Smithsonian American Art Museum in Washington, D.C., which bought the 

In [21]:
wikipediaapi.WikipediaPage('Q17141380')

TypeError: __init__() missing 1 required positional argument: 'title'

In [51]:
wikipedia.page(title='The_Headless_Horseman_Pursuing_Ichabod_Crane',
               pageid=None, auto_suggest=False, redirect=True, preload=False).content

'The Headless Horseman Pursuing Ichabod Crane (1858) is a painting by American artist John Quidor, depicting a scene from Washington Irving\'s short story "The Legend of Sleepy Hollow".\nThe schoolmaster Ichabod Crane is fleeing on a white horse, pursued by the Headless Horseman on a black horse.  In one hand, the Headless Horseman is holding a pumpkin, which he is preparing to throw at Crane.  Visible in the background is the Old Dutch Church of Sleepy Hollow and the adjacent Sleepy Hollow Cemetery, and the full moon.\nThe painting is one of several by the artist based on scenes from Irving\'s written works about Dutch New York, including such paintings as Ichabod Crane Flying from the Headless Horseman (1828, Yale University Art Gallery) and The Return of Rip Van Winkle (1849, National Gallery of Art).Quidor first exhibited the picture at the National Academy of Design in New York City. It is currently displayed in the Smithsonian American Art Museum in Washington, D.C., which bought

In [ ]:
zep = wikipedia.page(imago)

In [ ]:
def blerb(intext):
    zaz = intext.content.replace('\n\n\n== Description ==\n', ' ')
    zo = zaz.replace("\n\n\n== History ==", ' ')
    zo = zo.replace("\n\n\n== See also ==\n", ' ')
    zo = zo.replace("== Content ==", ".")
    zo = zo.replace("== Influences ==", ".")
    zo = zo.replace("== References ==", ".")
    zo = zo.replace("== Making ==", ".")

    zo = zo.replace("....", ".")
    zo = zo.rsplit('\n\n\n== Notes')[0]
    zo = zo.replace("\"", ".")
    zo = zo.replace("\n", ".")

    zo = zo.replace(".......", ".")
    zo = zo.replace("......", ".")
    zo = zo.replace(".....", ".")
    zo = zo.replace("....", ".")
    und = 0
    bruh = ""
    if len(zo) > 1500:
        for i in zo.rsplit('.'):
            if und < 1500:
                bruh += i
                und += len(i)
        bruh += '.'
    return bruh

In [ ]:
blerb(zep)

In [ ]:
wdir = '/Users/anthonycelio/Documents/deep_news/!machine/art_set/'

In [2]:
ga = 'https://en.wikipedia.org/wiki/Wikipedia:Featured_pictures/Artwork/Paintings'

In [3]:
req = Request(ga, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
soup = BeautifulSoup(webpage, "html.parser")

In [5]:
mydivs = soup.findAll("div", {"class": "gallerytext"})

In [6]:
imago = str(mydivs[0].findAll('a')[0]).rsplit(' ')[1][12:-1]
imagog = str(mydivs[0].findAll('a')[0]).rsplit(' ')[0][12:-1]
if len(imago) < len(imagog):
    imago = imagog

In [7]:
imago

'A_Philosopher_Lecturing_on_the_Orrery'

In [11]:
wikipedia.page('The_Headless_Horseman_Pursuing_Ichabod_Crane').content

'The Headless Horseman is a mythical figure who has appeared in folklore around the world since at least the Middle Ages. The Headless Horseman is traditionally depicted as a man upon horseback who is missing his head. Depending on the legend, the Horseman is either carrying his head, or is missing his head altogether, and is searching for it. Examples include the dullahan from Ireland who is a demonic fairy usually depicted riding a horse and carrying his head under his arm; the titular knight from the English tale Sir Gawain and the Green Knight; and "The Legend of Sleepy Hollow," a short story written in 1820 by American Washington Irving, which has been adapted into several other works of literature and film including the 1999 Tim Burton film Sleepy Hollow.\n\n\n== In American folklore ==\n\nThe Headless Horseman is a fictional character from the short story "The Legend of Sleepy Hollow" by American author Washington Irving. The story, from Irving\'s collection of short stories ent

In [ ]:
comment[0].findAll('p')[0].text

In [ ]:
len('In this interior scene of the artist’s summer residence in Maine, Frank Weston Benson depicted his daughterElisabeth curled up in a rattan chair near a large fireplace. A key figure of American Impressionism, Benson is best known for his outdoor figural compositions—often featuring his wife or children—in sun-filled, breezy landscapes. Rainy Day, in contrast, is a study of the subtler effects of interior light, with flat, overcast lighting filtering in from the left and embers glowing at far right. In the late 19th century, painters including Benson, Edmund Tarbell, and Thomas Wilmer Dewing composed scenes of women in interiors in a contemplative state or engaged in activities like sewing and reading, suggesting links between domesticity, femininity, and cultural refinement.')

In [ ]:
photo = soup.findAll("div", {"class": "PbVjBf uFenQ nqLMzf"})

In [ ]:
photo

In [ ]:
ga = 'https://artsandculture.google.com/asset/venus-and-vulcan/hAEuEwSrLZ_FQw?ms=%7B%22x%22%3A0.7040565848666179%2C%22y%22%3A0.33116513192360014%2C%22z%22%3A12%2C%22size%22%3A%7B%22width%22%3A0.12795477903391572%2C%22height%22%3A0.1456896551724138%7D%7D'

In [ ]:
req = Request(ga, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
soup = BeautifulSoup(webpage, "html.parser")

In [ ]:
urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', str(soup))

In [ ]:
urls

In [ ]:
wdir = '/Users/anthonycelio/Documents/deep_news/!machine/art_set/MET/'

In [ ]:
t = str(uuid.uuid4())[-12:]
sha = wdir+t+'/'
os.makedirs(sha)
os.chdir(sha)

In [ ]:
for m in urls:
    try:
        urllib.request.urlretrieve(m, sha+str(uuid.uuid4())[-5:]+'.png')
    except:
        pass

In [ ]:
d = {}
for i in os.listdir(sha):
    cu = os.stat(i).st_size
    d[i] = cu

In [ ]:
for x in os.listdir(sha):
    if x != max(d.items(), key=operator.itemgetter(1))[0]:
        if x[-4:] != 'ther':
            os.remove(x)

In [3]:
wdir = '/Users/anthonycelio/Documents/deep_news/!machine/art_set/artic/'

In [4]:
def get_art():
    inter = random.randint(0, 140000)
    ga = 'https://www.artic.edu/artworks/'+str(inter)+'/'
    #ga = 'https://www.artic.edu/artworks/64479/'
    req = Request(ga, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, "html.parser")
    comment = soup.findAll("div", {"class": "o-blocks"})

    cment = comment[0].findAll('p')[0].text
    if len(cment) > 650:
        wdf = pd.DataFrame([cment])

        t = str(uuid.uuid4())[-12:]
        sha = wdir+t+'/'
        os.makedirs(sha)
        os.chdir(sha)

        path = sha+'my_data.feather' 
        feather.write_dataframe(wdf, path)
        photo = soup.findAll("div", {"class": "m-article-header__img-container"})
        urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', str(photo))
        for m in urls:
            if m[-3:] == 'jpg':
                urllib.request.urlretrieve(m, sha+str(uuid.uuid4())[-5:]+'.jpg')
        d = {}
        for i in os.listdir(sha):
            cu = os.stat(i).st_size
            d[i] = cu
        for x in os.listdir(sha):
            if x != max(d.items(), key=operator.itemgetter(1))[0]:
                if x[-4:] != 'ther':
                    os.remove(x)

In [ ]:
get_art()

In [5]:
for i in tqdm(range(10000)):
    try:
        get_art()
    except:
        pass

In [ ]:
sha = '/Users/anthonycelio/Documents/deep_news/!machine/art_set/artic/'

In [ ]:
missed2 = []
for ii in tqdm(range(len(missed))):
    try:
         imago = str(mydivs[ii].findAll('a')[0]).rsplit(' ')[1][12:-1]
         imagog = str(mydivs[ii].findAll('a')[0]).rsplit(' ')[0][12:-1]
         if len(imago) < len(imagog):
             imago = imagog

        zep = wikipedia.page(imago)

        t = str(uuid.uuid4())[-12:]
        sha = wdir+t+'/'
        os.makedirs(sha)
        os.chdir(sha)
        for m in tqdm(zep.images):
            urllib.request.urlretrieve(m, sha+str(uuid.uuid4())[-5:]+m[-14:])
        d = {}
        for i in os.listdir(sha):
            cu = os.stat(i).st_size
            d[i] = cu
        for x in os.listdir(sha):
            if x != max(d.items(), key=operator.itemgetter(1))[0]:
                os.remove(x)

        wdf = pd.DataFrame([blerb(zep)])
        path = sha+'my_data.feather' 
        feather.write_dataframe(wdf, path)
    except:
        missed2.append(imago)
        pass

In [ ]:
zep = wikipedia.page(imago)

In [ ]:
wdf = pd.DataFrame([blerb(zep)])

In [ ]:
wdf

In [ ]:
path = wdir+'my_data.feather' 
feather.write_dataframe(wdf, path)
wdf = feather.read_dataframe(path)

In [ ]:
fea = '/Users/anthonycelio/Documents/deep_news/!machine/art_set/401c8e3bcb98/my_data.feather'

In [ ]:
furv = feather.read_dataframe(fea)

In [ ]:
furv.loc[0][0]

In [ ]:
shutil.rmtree

In [ ]:
for i in os.listdir(wdir):
    try:
        bub = wdir+i+'/'
        if len(os.listdir(bub)) != 2:
            shutil.rmtree(bub)
    except:
        print('I had to pass')
        pass

In [ ]:
Image.MAX_IMAGE_PIXELS = None
from PIL import Image
img = Image.open(emage)

In [ ]:
# W x H
new_img = img.resize((int(0.3 * img.width), int(0.3 * img.height)))
new_img.save(werdir+"resized.jpg", "JPEG", optimize=False)